In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import statistics as stat
from scipy.spatial.distance import cdist
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Event33.CSV", skiprows=[0,1,2,4]) # put whatever event 3 is
df

,Time,C_W919_W910_1_W910_A,C_W919_W910_1_W910_B,C_W919_W910_1_W910_C,C_W919_W910_1_W910_N,V_W-115KV2_AN,V_W-115KV2_BN,V_W-115KV2_CN,C_W957_W950_1_W950_A,C_W957_W950_1_W950_B,...,V_FD2B5_AN,V_FD2B5_BN,V_FD2B5_CN,C_FD3B4_FD3B5_1_FD3B4_A,C_FD3B4_FD3B5_1_FD3B4_B,C_FD3B4_FD3B5_1_FD3B4_C,C_FD3B4_FD3B5_1_FD3B4_N,V_FD3B4_AN,V_FD3B4_BN,V_FD3B4_CN
0,0,-0.000563,0.000422,0.001169,0.001686,-2.121724,-1.893156,0.199429,-0.000116,-0.000458,...,5.412048,0.325285,-0.169655,-0.003543,-0.000910,-0.002915,0.000972,1.273205,2.130958,-2.652985
1,208,9.111774,-4.179547,-5.028308,0.346913,34481.020000,-15795.660000,-17054.330000,45.886520,-22.153150,...,1543.987000,39.189480,-1575.474000,3.180644,-0.099283,-3.169328,0.026558,1921.046000,70.318860,-1967.699000
2,417,-23.319010,11.452000,11.412130,1.555206,151239.100000,-64842.840000,-81550.760000,10.535650,-2.858659,...,11757.510000,405.099600,-12657.540000,31.811110,1.041847,-33.354590,0.157332,12478.450000,666.963000,-13318.130000
3,625,-44.141320,19.288630,24.484140,1.653842,173384.000000,-66637.710000,-106158.400000,-19.376050,11.656900,...,14310.890000,1854.808000,-16386.210000,67.082090,5.517830,-73.519950,0.245462,14495.150000,2053.450000,-16671.270000
4,833,-53.516910,20.267810,33.157610,0.420890,99366.700000,-25844.770000,-82634.670000,-60.227240,29.399120,...,9313.247000,3248.786000,-12470.770000,82.132140,13.082730,-96.047760,0.171546,8625.440000,3269.468000,-11924.310000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96000,19999872,-45.416350,35.087120,10.330530,-0.022973,93532.110000,-52015.450000,-41572.780000,-22.346420,19.634140,...,10087.990000,-1365.788000,-8705.755000,101.210000,-30.644180,-70.534990,-0.001871,10057.870000,-1344.580000,-8708.264000
96001,20000080,-46.355180,32.428890,13.930740,-0.096950,93706.900000,-45738.780000,-48024.820000,-23.051090,18.450310,...,9716.986000,-507.109200,-9194.235000,99.032820,-22.744810,-76.232720,-0.007557,9688.989000,-486.549000,-9193.813000
96002,20000289,-47.008550,29.568400,17.448540,-0.170928,93309.090000,-39182.680000,-54178.450000,-23.611400,17.153110,...,9291.423000,354.502800,-9626.091000,96.256860,-14.697870,-81.467740,-0.010400,9258.184000,371.482100,-9622.239000
96003,20000497,-47.373300,26.518500,20.863330,-0.220246,92326.620000,-32379.380000,-59999.350000,-24.027350,15.749080,...,8811.300000,1213.915000,-9998.790000,92.882170,-6.562406,-86.194680,-0.016086,8773.529000,1229.513000,-9993.541000


In [4]:
# remove first 0.25 seconds to allow system to stabilise before using data
data = df[df['Time']> 0.25e6].iloc[:, 1:]
data

,C_W919_W910_1_W910_A,C_W919_W910_1_W910_B,C_W919_W910_1_W910_C,C_W919_W910_1_W910_N,V_W-115KV2_AN,V_W-115KV2_BN,V_W-115KV2_CN,C_W957_W950_1_W950_A,C_W957_W950_1_W950_B,C_W957_W950_1_W950_C,...,V_FD2B5_AN,V_FD2B5_BN,V_FD2B5_CN,C_FD3B4_FD3B5_1_FD3B4_A,C_FD3B4_FD3B5_1_FD3B4_B,C_FD3B4_FD3B5_1_FD3B4_C,C_FD3B4_FD3B5_1_FD3B4_N,V_FD3B4_AN,V_FD3B4_BN,V_FD3B4_CN
1201,-42.15266,26.08831,16.06820,-0.146268,93628.55,-41713.77,-51966.02,-20.59943,15.210580,5.408786,...,9466.013,75.12054,-9522.142,97.53598,-17.383460,-80.06145,-0.010400,9433.199,93.04816,-9519.417
1202,-42.50154,23.37550,19.13276,-0.195587,92869.09,-34985.70,-57927.56,-21.00070,14.007130,7.023959,...,8996.802,938.93240,-9915.123,94.32458,-9.247996,-84.97892,-0.016086,8964.699,956.76150,-9907.856
1203,-42.58718,20.51502,22.08400,-0.269564,91530.99,-28048.05,-63528.94,-21.27474,12.718650,8.596285,...,8473.032,1796.87800,-10247.260,90.56887,-1.043666,-89.36109,-0.021772,8442.348,1814.79300,-10239.170
1204,-42.41274,17.52290,24.90133,-0.318882,89632.34,-20933.06,-68742.71,-21.41665,11.349500,10.115380,...,7905.614,2643.82400,-10516.000,86.21443,7.160663,-93.19889,-0.024614,7866.147,2655.77700,-10507.650
1205,-41.97504,14.42485,27.56672,-0.368200,87179.16,-13694.48,-73531.15,-21.42644,9.908409,11.572150,...,7272.725,3474.63800,-10719.680,81.34289,15.325640,-96.45603,-0.030300,7241.481,3485.39600,-10707.580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96000,-45.41635,35.08712,10.33053,-0.022973,93532.11,-52015.45,-41572.78,-22.34642,19.634140,2.722459,...,10087.990,-1365.78800,-8705.755,101.21000,-30.644180,-70.53499,-0.001871,10057.870,-1344.58000,-8708.264
96001,-46.35518,32.42889,13.93074,-0.096950,93706.90,-45738.78,-48024.82,-23.05109,18.450310,4.616781,...,9716.986,-507.10920,-9194.235,99.03282,-22.744810,-76.23272,-0.007557,9688.989,-486.54900,-9193.813
96002,-47.00855,29.56840,17.44854,-0.170928,93309.09,-39182.68,-54178.45,-23.61140,17.153110,6.483837,...,9291.423,354.50280,-9626.091,96.25686,-14.697870,-81.46774,-0.010400,9258.184,371.48210,-9622.239
96003,-47.37330,26.51850,20.86333,-0.220246,92326.62,-32379.38,-59999.35,-24.02735,15.749080,8.311942,...,8811.300,1213.91500,-9998.790,92.88217,-6.562406,-86.19468,-0.016086,8773.529,1229.51300,-9993.541


In [5]:
test_path = "Attack_Faults_001.csv"
df_test = pd.read_csv("/content/drive/MyDrive/Event43.CSV", skiprows=[0,1,2,4]) # put whatever event 1 is

df_test

,Time,C_W919_W910_1_W910_A,C_W919_W910_1_W910_B,C_W919_W910_1_W910_C,C_W919_W910_1_W910_N,V_W-115KV2_AN,V_W-115KV2_BN,V_W-115KV2_CN,C_W957_W950_1_W950_A,C_W957_W950_1_W950_B,...,V_FD2B5_AN,V_FD2B5_BN,V_FD2B5_CN,C_FD3B4_FD3B5_1_FD3B4_A,C_FD3B4_FD3B5_1_FD3B4_B,C_FD3B4_FD3B5_1_FD3B4_C,C_FD3B4_FD3B5_1_FD3B4_N,V_FD3B4_AN,V_FD3B4_BN,V_FD3B4_CN
0,0,-0.000047,0.000086,-0.000928,0.000327,1.5112,1.134644,-0.935609,0.000071,0.000389,...,-0.026282,-0.052717,0.062506,0.001473,-0.001534,-0.000641,0.000005,-0.003542,0.047273,-0.133261
1,208,9.112645,-4.181053,-5.029214,0.355775,34481.5000,-15794.320000,-17055.280000,45.885850,-22.153730,...,1546.851000,39.433820,-1575.525000,3.177475,-0.097298,-3.167078,0.027795,1919.898000,67.809930,-1965.721000
2,417,-23.320950,11.453140,11.411820,1.543984,151241.7000,-64842.910000,-81549.510000,10.534790,-2.858789,...,11758.830000,404.937400,-12657.170000,31.799680,1.045492,-33.355310,0.158221,12478.360000,668.616400,-13320.010000
3,625,-44.141020,19.289100,24.485370,1.647103,173384.0000,-66639.040000,-106159.000000,-19.375650,11.657370,...,14308.390000,1854.465000,-16386.210000,67.085770,5.520887,-73.520630,0.244365,14495.150000,2052.383000,-16668.640000
4,833,-53.516520,20.266240,33.158640,0.430770,99362.0800,-25843.330000,-82635.040000,-60.227240,29.400110,...,9312.693000,3248.981000,-12470.800000,82.125630,13.086280,-96.044830,0.170725,8624.661000,3270.769000,-11922.220000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96000,19999872,-42.781100,32.253990,10.527180,-0.077012,93607.8200,-51984.720000,-41677.240000,-20.740660,17.928120,...,10095.960000,-1333.483000,-8746.760000,101.366200,-30.397190,-70.934100,-0.001715,10065.280000,-1309.583000,-8748.536000
96001,20000080,-43.594550,29.718890,13.880060,-0.137164,93777.7900,-45699.730000,-48131.330000,-21.370540,16.820750,...,9725.167000,-472.541200,-9234.154000,99.164020,-22.464700,-76.627760,-0.006669,9693.214000,-451.144600,-9233.294000
96002,20000289,-44.141020,26.992970,17.153670,-0.194973,93371.5100,-39132.360000,-54286.820000,-21.867650,15.607560,...,9293.819000,391.100300,-9664.845000,96.350810,-14.394940,-81.853530,-0.011556,9261.187000,410.313200,-9660.724000
96003,20000497,-44.418420,24.101400,20.325030,-0.251220,92384.8300,-32323.370000,-60109.500000,-22.230360,14.298000,...,8805.455000,1252.379000,-10035.500000,92.939310,-6.235804,-86.571880,-0.016356,8771.979000,1269.087000,-10028.720000


In [6]:
data_test = df_test[df_test['Time']> 0.25e6].iloc[:, 1:]

data_test

,C_W919_W910_1_W910_A,C_W919_W910_1_W910_B,C_W919_W910_1_W910_C,C_W919_W910_1_W910_N,V_W-115KV2_AN,V_W-115KV2_BN,V_W-115KV2_CN,C_W957_W950_1_W950_A,C_W957_W950_1_W950_B,C_W957_W950_1_W950_C,...,V_FD2B5_AN,V_FD2B5_BN,V_FD2B5_CN,C_FD3B4_FD3B5_1_FD3B4_A,C_FD3B4_FD3B5_1_FD3B4_B,C_FD3B4_FD3B5_1_FD3B4_C,C_FD3B4_FD3B5_1_FD3B4_N,V_FD3B4_AN,V_FD3B4_BN,V_FD3B4_CN
1201,-42.15329,26.08922,16.06872,-0.144976,93628.55,-41711.56,-51966.46,-20.59979,15.210910,5.409099,...,9466.830,74.87046,-9521.837,97.53147,-17.385990,-80.06438,-0.009649,9434.316,94.98208,-9519.090
1202,-42.50161,23.37379,19.13368,-0.205910,92865.73,-34986.99,-57925.99,-21.00136,14.008040,7.023896,...,8999.306,938.84940,-9915.004,94.33637,-9.242809,-84.97713,-0.015216,8965.359,956.44000,-9909.426
1203,-42.58713,20.51367,22.08389,-0.265282,91530.81,-28047.01,-63530.94,-21.27501,12.717410,8.595996,...,8475.948,1796.75400,-10246.880,90.55890,-1.045367,-89.36270,-0.020684,8441.606,1812.19500,-10238.640
1204,-42.41192,17.52355,24.90057,-0.323872,89632.07,-20935.27,-68740.87,-21.41750,11.348480,10.115150,...,7900.292,2643.85800,-10515.800,86.22134,7.161652,-93.19801,-0.026032,7865.437,2656.54500,-10505.050
1205,-41.97600,14.42439,27.56702,-0.379337,87181.95,-13692.53,-73530.89,-21.42722,9.908994,11.572830,...,7275.879,3474.42500,-10719.680,81.35550,15.323980,-96.45670,-0.031215,7240.823,3484.79200,-10706.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96000,-42.78110,32.25399,10.52718,-0.077012,93607.82,-51984.72,-41677.24,-20.74066,17.928120,2.819958,...,10095.960,-1333.48300,-8746.760,101.36620,-30.397190,-70.93410,-0.001715,10065.280,-1309.58300,-8748.536
96001,-43.59455,29.71889,13.88006,-0.137164,93777.79,-45699.73,-48131.33,-21.37054,16.820750,4.566262,...,9725.167,-472.54120,-9234.154,99.16402,-22.464700,-76.62776,-0.006669,9693.214,-451.14460,-9233.294
96002,-44.14102,26.99297,17.15367,-0.194973,93371.51,-39132.36,-54286.82,-21.86765,15.607560,6.285239,...,9293.819,391.10030,-9664.845,96.35081,-14.394940,-81.85353,-0.011556,9261.187,410.31320,-9660.724
96003,-44.41842,24.10140,20.32503,-0.251220,92384.83,-32323.37,-60109.50,-22.23036,14.298000,7.966644,...,8805.455,1252.37900,-10035.500,92.93931,-6.235804,-86.57188,-0.016356,8771.979,1269.08700,-10028.720


In [7]:
attack_dict = {15: (2.20, 3.50, 'CT', ' C_W957_W950_1_W957_A'),
                   16: (2.20, 3.50, 'CT', ' C_W957_W950_1_W957_B'),
                   17: (2.20, 3.50, 'CT', ' C_W957_W950_1_W957_C'),
                   3: (3.80, 4.50, 'Cphase', 'C_W919_W910_1_W910_C'),
                   48: (4.5, 5.9, 'PT', ' V_W9111_AN'),
                   49: (4.5, 5.9, 'PT', ' V_W9111_BN'),
                   50: (4.5, 5.9, 'PT', ' V_W9111_CN'),
                   42: (6.2, 7.1, 'Vphase', ' V_BUS13KV1_BN'),
                   30: (8.5, 9.4, 'GPS', ' C_W919_W910_1_W919'),
                   31: (8.5, 9.4, 'GPS', ' C_W919_W910_1_W919'),
                   32: (8.5, 9.4, 'GPS', ' C_W919_W910_1_W919'),
                   # 33: (8.5, 9.4, 'GPS', 'C_CTWE1&V_PTWE1'),
                   34: (8.5, 9.4, 'GPS', 'V_W911_'),
                   35: (8.5, 9.4, 'GPS', 'V_W911_'),
                   36: (8.5, 9.4, 'GPS', 'V_W911_'),
                   80: (10.5, 11.5, 'PT2', 'V_SOURCE-FA_AN'),
                   81: (10.5, 11.5, 'PT2', 'V_SOURCE-FA_BN'),
                   82: (10.5, 11.5, 'PT2', 'V_SOURCE-FA_CN'),
                   111: (11.7, 12.7, 'CT2', 'C_FD2B2_FD2B1_1_FD2B1_A'),
                   112: (11.7, 12.7, 'CT2', 'C_FD2B2_FD2B1_1_FD2B1_B'),
                   113: (11.7, 12.7, 'CT2', 'C_FD2B2_FD2B1_1_FD2B1_C'),
                   # 114: (11.7, 12.7, 'CT', 'C_FD2B2_FD2B1_1_FD2B1_N')
                   120: (14.6, 15.8, 'Cphase2', ' C_FD2B5_FD2B6_1_FD2B5_C'),
                   97: (16.5, 17.5, 'GPS2', ' C_W787_W788_2_W787_ &  V_W788_'),
                   98: (16.5, 17.5, 'GPS2', ' C_W787_W788_2_W787_ &  V_W788_'),
                   99: (16.5, 17.5, 'GPS2', ' C_W787_W788_2_W787_ &  V_W788_'),
                   #100: (16.5, 17.5, 'GPS2', ' C_W787_W788_2_W787_ &  V_W788_'),
                   101: (16.5, 17.5, 'GPS2', ' C_W787_W788_2_W787_ &  V_W788_'),
                   102: (16.5, 17.5, 'GP2', ' C_W787_W788_2_W787_ &  V_W788_'),
                   103: (16.5, 17.5, 'GPS2', ' C_W787_W788_2_W787_ &  V_W788_'),
                   }

In [8]:

all_attack_times = set()
for k, values in attack_dict.items():
    all_attack_times.add((values[0], values[1]))



In [9]:
cols = data.columns

print(all_attack_times)

all_attack_indices = set()

for start, end in all_attack_times:
    attack_indices = df_test.index[(df_test['Time'] >= start * 1e6) & (df_test['Time'] <= end * 1e6)] - 1201
    all_attack_indices.add((attack_indices[0], attack_indices[-1]))

all_attack_indice = sorted(list(all_attack_indices))

all_attack_indices

{(6.2, 7.1), (3.8, 4.5), (14.6, 15.8), (2.2, 3.5), (4.5, 5.9), (8.5, 9.4), (16.5, 17.5), (11.7, 12.7), (10.5, 11.5)}


{(9360, 15599),
 (17040, 20399),
 (20400, 27119),
 (28560, 32879),
 (39600, 43919),
 (49200, 53999),
 (54960, 59759),
 (68880, 74639),
 (78000, 82799)}

In [10]:
adj_list =  {69: [26, 15],
                     70: [27, 16],
                     71: [28, 17],
                     72: [29, 18],
                     76: [8],
                     77: [9],
                     78: [10],
                     79: [11],
                     62: [30, 19],
                     63: [31, 20],
                     64: [32, 21],
                     65: [33, 22],
                     30: [62, 1, 44],
                     31: [63, 2, 45],
                     32: [64, 3, 46],
                     33: [65, 4, 47],
                     26: [69, 19],
                     27: [70, 20],
                     28: [71, 21],
                     29: [72, 22],
                     19: [62, 26],
                     20: [63, 27],
                     21: [64, 28],
                     22: [65, 29],
                     8: [76, 15],
                     9: [77, 16],
                     10: [78, 17],
                     11: [79, 18],
                     1: [30],
                     2: [31],
                     3: [32],
                     4: [33],
                     15: [69, 8],
                     16: [70, 9],
                     17: [71, 10],
                     18: [72, 11],
                     51: [55],
                     52: [56],
                     53: [57],
                     54: [58],
                     44: [30, 37],
                     45: [31, 38],
                     46: [32, 39],
                     47: [33, 40],
                     55: [51, 83, 90, 97, 104],
                     56: [52, 84, 91, 98, 105],
                     57: [53, 85, 92, 99, 106],
                     58: [54, 86, 93, 100, 107],
                     37: [44],
                     38: [45],
                     39: [46],
                     40: [47],
                     83: [55],
                     84: [56],
                     85: [57],
                     86: [58],
                     90: [55, 111],
                     91: [56, 112],
                     92: [57, 113],
                     93: [58, 114],
                     111: [90, 118],
                     112: [91, 119],
                     113: [92, 120],
                     114: [93, 121],
                     118: [111],
                     119: [112],
                     120: [113],
                     121: [114],
                     97: [55, 125],
                     98: [56, 126],
                     99: [57, 127],
                     100: [58, 128],
                     125: [97],
                     126: [98],
                     127: [99],
                     128: [100],
                     104: [55],
                     105: [56],
                     106: [57],
                     107: [58],
                     73: [23],
                     74: [24],
                     75: [25],
                     80: [12],
                     81: [13],
                     82: [14],
                     66: [34],
                     67: [35],
                     68: [36],
                     34: [66, 23, 12, 5, 48],
                     35: [67, 24, 13, 6, 49],
                     36: [68, 25, 14, 7, 50],
                     23: [73, 34, 12, 5, 48],
                     24: [74, 35, 13, 6, 49],
                     25: [75, 36, 14, 7, 50],
                     12: [80, 34, 23, 5, 48],
                     13: [81, 35, 24, 6, 49],
                     14: [82, 36, 25, 7, 50],
                     5: [34, 23, 12, 48, 59],
                     6: [35, 24, 13, 49, 60],
                     7: [36, 25, 14, 50, 61],
                     48: [34, 23, 12, 5, 41],
                     49: [35, 24, 13, 6, 42],
                     50: [36, 25, 14, 7, 43],
                     59: [5, 87, 94, 101, 108],
                     60: [6, 88, 95, 102, 109],
                     61: [7, 89, 96, 103, 110],
                     41: [48],
                     42: [49],
                     43: [50],
                     87: [59],
                     88: [60],
                     89: [61],
                     94: [59, 115],
                     95: [60, 116],
                     96: [61, 117],
                     115: [94, 122],
                     116: [95, 123],
                     117: [96, 124],
                     122: [115],
                     123: [116],
                     124: [117],
                     101: [59, 129],
                     102: [60, 130],
                     103: [61, 131],
                     129: [101],
                     130: [102],
                     131: [103],
                     108: [59],
                     109: [60],
                     110: [61]}

In [11]:
neutral_sensors = [4,11,18, 22, 29, 33,40, 47, 54, 58, 65, 72, 79, 86, 93, 100, 107, 114, 121, 128]
emptyr = []  # nodes without connections to anything
unmonitored_sensors = set(neutral_sensors + emptyr)

new_adj_list = {}
for sensors, neigh in adj_list.items():
  if sensors in unmonitored_sensors:
    continue
  else:
    new_nei = []
    for i in neigh:
      if i in unmonitored_sensors:
        continue
      else:
        new_nei += [i]
    new_adj_list[sensors] = new_nei

adj_list = new_adj_list

In [12]:
attack_list = ['CT', 'Cphase', 'PT', 'Vphase', 'GPS','PT2', 'CT2', 'Cphase2', 'GPS2']


In [13]:
attacked_sensors = []
for index, i in enumerate(attack_list):
  attak_list = []
  for j, value in attack_dict.items():
    if value[2] == i:
      attak_list += [j]+adj_list[j]
      print(attak_list)
  attacked_sensors += [attak_list]
attacked_sensors

[15, 69, 8]
[15, 69, 8, 16, 70, 9]
[15, 69, 8, 16, 70, 9, 17, 71, 10]
[3, 32]
[48, 34, 23, 12, 5, 41]
[48, 34, 23, 12, 5, 41, 49, 35, 24, 13, 6, 42]
[48, 34, 23, 12, 5, 41, 49, 35, 24, 13, 6, 42, 50, 36, 25, 14, 7, 43]
[42, 49]
[30, 62, 1, 44]
[30, 62, 1, 44, 31, 63, 2, 45]
[30, 62, 1, 44, 31, 63, 2, 45, 32, 64, 3, 46]
[30, 62, 1, 44, 31, 63, 2, 45, 32, 64, 3, 46, 34, 66, 23, 12, 5, 48]
[30, 62, 1, 44, 31, 63, 2, 45, 32, 64, 3, 46, 34, 66, 23, 12, 5, 48, 35, 67, 24, 13, 6, 49]
[30, 62, 1, 44, 31, 63, 2, 45, 32, 64, 3, 46, 34, 66, 23, 12, 5, 48, 35, 67, 24, 13, 6, 49, 36, 68, 25, 14, 7, 50]
[80, 12]
[80, 12, 81, 13]
[80, 12, 81, 13, 82, 14]
[111, 90, 118]
[111, 90, 118, 112, 91, 119]
[111, 90, 118, 112, 91, 119, 113, 92, 120]
[120, 113]
[97, 55, 125]
[97, 55, 125, 98, 56, 126]
[97, 55, 125, 98, 56, 126, 99, 57, 127]
[97, 55, 125, 98, 56, 126, 99, 57, 127, 101, 59, 129]
[97, 55, 125, 98, 56, 126, 99, 57, 127, 101, 59, 129, 103, 61, 131]


[[15, 69, 8, 16, 70, 9, 17, 71, 10],
 [3, 32],
 [48, 34, 23, 12, 5, 41, 49, 35, 24, 13, 6, 42, 50, 36, 25, 14, 7, 43],
 [42, 49],
 [30,
  62,
  1,
  44,
  31,
  63,
  2,
  45,
  32,
  64,
  3,
  46,
  34,
  66,
  23,
  12,
  5,
  48,
  35,
  67,
  24,
  13,
  6,
  49,
  36,
  68,
  25,
  14,
  7,
  50],
 [80, 12, 81, 13, 82, 14],
 [111, 90, 118, 112, 91, 119, 113, 92, 120],
 [120, 113],
 [97, 55, 125, 98, 56, 126, 99, 57, 127, 101, 59, 129, 103, 61, 131]]

# Method

Get the anomalous data, get K-Nearest distance from anomolous data to training data. Divide that by median and get the G-score. Collect the G-score for all sensors. Use percentiles to find out which sensor is attacked.

In [14]:
def medians (dist):
  #finds the median of each dist
  median_list = []
  for i in dist:
    medi = np.median(i)
    median_list += [medi]
  return median_list

**Test for all**

**Thresholding and checking every sensor**

In [15]:
col = data.columns

In [16]:
#count of sensors in total attack neighborhood
c = 0
for i in attacked_sensors:
  for sensors in i:
    c += 1

In [17]:
def g_scoring (dist_sensor, median_list):
  g_scores = []
  for k in [1,5]:
    k = int(k)
    dist_100 = np.partition(dist_sensor[:], kth=k-1, axis=1)[:,k-1]
    g_score = np.divide(dist_100, median_list)
    g_scores += [g_score]
  return g_scores

In [18]:
def multi_gscoring (g_scores, g_score1, g_score5):
  for ind, i in enumerate(g_scores):
    g_mean = stat.mean(i)
    if ind == 0:
      g_score1 += [g_mean]
    if ind == 1:
      g_score5 += [g_mean]
    #if ind == 2:
      #g_score10 += [g_mean]
    #if ind == 3:
      #g_score50 += [g_mean]
    #if ind == 4:
      #g_score100 += [g_mean]

In [19]:
def threshold_gscore (quantile, g_score1, g_score5):
  g_score1.sort()
  g_score5.sort()
  threshold1 = g_score1[-quantile]
  threshold5 = g_score5[-quantile]
  #threshold10 = np.quantile(g_score10, quantile)
  #threshold50 = np.quantile(g_score50, quantile)
  #threshold100 = np.quantile(g_score100, quantile)
  return threshold1, threshold5

Calculate LD (Localization Delay) FPR-L (False Positive Rate Localization) and FLR (False Localization Rate) and Time to run a window.

In [20]:
import time
start_time = time.time()
co = 0
flr = []
fpr_d = []
fpr_l = []
for start, end in sorted(all_attack_indice):
  g_score1 = []
  g_score5 = []
  g_score_dict = {}

  #calculating g_scores
  for index, columns in enumerate(col):
    sensor = index+1
    if sensor in unmonitored_sensors:
        continue #taking out empty rows
    else:
      nbrs = np.array(adj_list[sensor]+[sensor]) -1
      data_sensor=data.iloc[start:end, nbrs]
      data_test_sensor=data_test.iloc[start:end, nbrs]
      dist_sensor= cdist(data_sensor, data_test_sensor) #cdisting the whole thing
      median_list = medians(dist_sensor)
      g_scores = g_scoring(dist_sensor, median_list)
      multi_gscoring (g_scores, g_score1, g_score5)
      g_score_dict[sensor] = g_scores

  #thresholding
  num_sen = len(attacked_sensors[co])
  threshold1, threshold5 = threshold_gscore (num_sen, g_score1, g_score5)

  #finding the anomalies that crosses the thresholds
  anomaly1 = [i for i in g_score1 if i>= threshold1]
  anomaly5 = [i for i in g_score5 if i>= threshold5]
  #converting them to names
  anomaly1_names = []
  anomaly5_names = []

  for key, value in g_score_dict.items():
    for i in value:
      j=stat.mean(i)
      if j in anomaly1:
        anomaly1_names += [key]
      if j in anomaly5:
        anomaly5_names += [key]

  false_localised_sensors = []
  for anom in anomaly5_names:
    if anom in attacked_sensors[co]:
      continue
    else:
      false_localised_sensors += [anom]

  #FLR
  print(f"For {attack_list[co]} attack, the sensors that are detected to be anomalous is:")
  print(f"K=1 {anomaly1_names}")
  print(f"K=5 {anomaly5_names}")

  print(f"1 Hop: {attacked_sensors[co]}")
  print(f"False Localisations: {false_localised_sensors}")

  flr.append(len(false_localised_sensors)/num_sen)

  #FPR_L
  timesteps = end-start
  false_positives = len(false_localised_sensors) * timesteps
  total_negatives = timesteps * (len(cols)-len(attacked_sensors[co]))
  FPR_L = false_positives / total_negatives
  fpr_l.append(FPR_L)

  #FPR_D
  total_t = 94804
  negatives = total_t *(len(cols)-len(attacked_sensors[co]))
  FPR_D = false_positives/negatives
  fpr_d.append(FPR_D)

  #LD

  co += 1

total_time = time.time() - start_time

For CT attack, the sensors that are detected to be anomalous is:
K=1 [19, 20, 21, 37, 38, 39, 44, 45, 46]
K=5 [19, 20, 21, 37, 38, 39, 44, 45, 46]
1 Hop: [15, 69, 8, 16, 70, 9, 17, 71, 10]
False Localisations: [19, 20, 21, 37, 38, 39, 44, 45, 46]
For Cphase attack, the sensors that are detected to be anomalous is:
K=1 [32, 44]
K=5 [32, 44]
1 Hop: [3, 32]
False Localisations: [44]
For PT attack, the sensors that are detected to be anomalous is:
K=1 [19, 20, 21, 26, 27, 28, 37, 38, 39, 44, 45, 46, 69, 70, 71, 125, 126, 127]
K=5 [19, 20, 21, 26, 27, 28, 37, 38, 39, 44, 45, 46, 69, 70, 71, 125, 126, 127]
1 Hop: [48, 34, 23, 12, 5, 41, 49, 35, 24, 13, 6, 42, 50, 36, 25, 14, 7, 43]
False Localisations: [19, 20, 21, 26, 27, 28, 37, 38, 39, 44, 45, 46, 69, 70, 71, 125, 126, 127]
For Vphase attack, the sensors that are detected to be anomalous is:
K=1 [44, 45]
K=5 [44, 45]
1 Hop: [42, 49]
False Localisations: [44, 45]
For GPS attack, the sensors that are detected to be anomalous is:
K=1 [1, 2, 

In [29]:
(np.mean(fpr_d)*100, np.std(fpr_l)*100)

(0.387771046215104, 5.957239579991392)

In [30]:
(np.mean(fpr_l)*100,np.std(fpr_l)*100)

(6.947468577575927, 5.957239579991392)

In [22]:
flr

[1.0, 0.5, 1.0, 1.0, 0.6, 1.0, 1.0, 0.0, 0.5333333333333333]

In [23]:
np.mean(flr), np.std(flr)


(0.7370370370370369, 0.3342374569682512)

In [24]:
time_a = 0
for star, en in all_attack_indices:
  time_a += (en-star)
time_a

45111

In [25]:
total_time*1000/time_a

17.823072017931736